In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns

import torch
import torch.nn as nn
import torchvision

from time import sleep
from sklearn.model_selection import train_test_split

In [2]:
DATASET_FOLDER_PATH = '/Users/aref/dvs-dataset/DvsGesture'

nb_image_height = 64
nb_image_weight = 64
nb_inputs  = nb_image_height*nb_image_weight
nb_hidden  = 100
nb_outputs = 12

time_step = 1e-3
nb_steps  = 200

batch_size = 16

In [3]:
dtype = torch.float

# Check whether a GPU is available
if torch.cuda.is_available():
    print('using cuda...')
    device = torch.device("cuda")     
else:
    print('using cpu...')
    device = torch.device("cpu")

using cpu...


In [4]:
# Load file list:
train_trail_file = 'trials_to_train.txt'
test_trail_file = 'trials_to_test.txt'

def load_trail_files(trail_file):
    file_list = []
    with open(os.path.join(DATASET_FOLDER_PATH, trail_file), 'r') as f:
        for line in f.readlines():
            aedat_file = line.strip()
            if not line:
                continue
            csv_file = aedat_file.replace('.aedat', '_labels.csv')
            file_list.append((
                os.path.join(DATASET_FOLDER_PATH, aedat_file),
                os.path.join(DATASET_FOLDER_PATH, csv_file)
            ))
    return file_list

file_list_train = load_trail_files(train_trail_file)
file_list_test = load_trail_files(test_trail_file)

def get_label(timestamp):
    for t in event_labels.keys():
        if t > timestamp:
            return event_labels[t]
    return 0

def get_label_text(timestamp):
    return gesture_mapping[get_label(timestamp)]

# mapping
gesture_mapping = {
    0: 'no_gesture',
    1: 'hand_clapping',
    2: 'right_hand_wave',
    3: 'left_hand_wave',
    4: 'right_arm_clockwise',
    5: 'right_arm_counter_clockwise',
    6: 'left_arm_clockwise',
    7: 'left_arm_counter_clockwise',
    8: 'arm_roll',
    9: 'air_drums',
    10: 'air_guitar',
    11: 'other_gestures',
}

In [5]:
from reader import read_file_all
from collections import OrderedDict

def read_event_labels(path):
    label_began_time = None
    event_labels = OrderedDict()
    with open(path, 'r') as f:
        for line in f.readlines():
            label, start, end = line.strip().split(',')
#             print '%5s\t%15s\t%15s' % (label, start, end)
            try:
                label = int(label)
                start = int(start)
                end = int(end)
            except ValueError:
                continue
            if label_began_time is None:
                label_began_time = start

            event_labels[start] = 0
            event_labels[end] = label
    return event_labels, label_began_time


def cache_trail_set_from_file_list(trail, file_list):
    counter = 0
    for file_name_set in file_list:
        x_train = np.zeros([0, 100, nb_inputs])
        y_train = []
        counter += 1
        print '%d out of %d ----------------------------------------------------------' % (counter, len(file_list))
        print '    reading "%s"' % file_name_set[0]
        event_labels, label_began_time = read_event_labels(file_name_set[1])
        event_list = read_file_all(file_name_set[0])

        print('    processing file')
        img = np.zeros([nb_image_weight, nb_image_height])

        max_time = None
        min_time = None
        begin_time = None
        end_time = None

        prev_label = None
        frame_counter = 0
        x_frame = np.zeros([1, 100, nb_inputs])
        for e in event_list:
            if e != 'clear':
                img[int(e['y']/2), int(e['x']/2)] = 1 # if e['polarity'] == 1 else 0
                if e['timestamp'] < min_time or min_time is None:
                    min_time = e['timestamp']
                if e['timestamp'] > max_time or max_time is None:
                    max_time = e['timestamp']
            else:
                if begin_time is None:
                    begin_time = min_time
                end_time = max_time
                mean_time = (max_time + min_time)/2
                mean_time = mean_time - begin_time + label_began_time
                label = get_label(mean_time)
                max_time = None
                min_time = None
                if prev_label is None:
                    prev_label = label
                elif prev_label != label:
                    prev_label = None
                    x_frame = np.zeros([1, 100, nb_inputs])
                    img = np.zeros([nb_image_weight, nb_image_height])
                    frame_counter = 0
                    continue

                if frame_counter == 100:
                    y_train.append(label)
                    x_train = np.append(x_train, x_frame, axis=0)
                    x_frame = np.zeros([1, 100, nb_inputs])
                    frame_counter = 0
                else:
                    x_frame[0, frame_counter, :] = img.flatten()
                    frame_counter += 1
                img = np.zeros([nb_image_weight, nb_image_height])

        y_train = np.array(y_train)
        print '    ', x_train.shape, ' ~> ', y_train.shape
        np.save(file="x_%s_cache_%d" % (trail, counter), arr=x_train)
        np.save(file="y_%s_cache_%d" % (trail, counter), arr=y_train)
        print '    saved!'
        
cache_trail_set_from_file_list('train', file_list_train)
cache_trail_set_from_file_list('test', file_list_test)

1 out of 98 ----------------------------------------------------------
    reading "/Users/aref/dvs-dataset/DvsGesture/user01_fluorescent.aedat"
loading dataset...
done!
    processing file


NameError: global name 'event_labels' is not defined

In [ ]:
def from_cache_generator(train_set=True):
    counter = 0
    file_list = file_list_train if train_set else file_list_test
    for file_name_set in file_list:
        counter += 1
        x_train = np.load(file="x_train_cache_%d.npy" % counter)
        y_train = np.load(file="y_train_cache_%d.npy" % counter)
        yield x_train, y_train

In [ ]:
tau_mem = 10e-3
tau_syn = 5e-3

alpha   = float(np.exp(-time_step/tau_syn))
beta    = float(np.exp(-time_step/tau_mem))

In [ ]:
weight_scale = 7*(1.0-beta) # this should give us some spikes to begin with

w1 = torch.empty((nb_inputs, nb_hidden),  device=device, dtype=dtype, requires_grad=True)
torch.nn.init.normal_(w1, mean=0.0, std=weight_scale/np.sqrt(nb_inputs))

w2 = torch.empty((nb_hidden, nb_outputs), device=device, dtype=dtype, requires_grad=True)
torch.nn.init.normal_(w2, mean=0.0, std=weight_scale/np.sqrt(nb_hidden))

loss_hist = []

print('init')

In [ ]:
class SuperSpike(torch.autograd.Function):
    scale = 50.0 # controls steepness of surrogate gradient

    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        out = torch.zeros_like(input)
        out[input > 0] = 1.0
        return out

    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        return grad_input/(SuperSpike.scale*torch.abs(input)+1.0)**2
#         return grad_input
    
spike_fn  = SuperSpike.apply

In [ ]:
def run_snn(inputs):
    h1 = torch.einsum("abc,cd->abd", (inputs, w1))
    syn = torch.zeros((batch_size,nb_hidden), device=device, dtype=dtype)
    mem = torch.zeros((batch_size,nb_hidden), device=device, dtype=dtype)

    mem_rec = [mem]
    spk_rec = [mem]

    # Compute hidden layer activity
    for t in range(nb_steps):
        mthr = mem-1.0
        out = spike_fn(mthr)
        rst = torch.zeros_like(mem)
        c   = (mthr > 0)
        rst[c] = torch.ones_like(mem)[c]

        new_syn = alpha*syn +h1[:,t]
        new_mem = beta*mem +syn -rst

        mem = new_mem
        syn = new_syn

        mem_rec.append(mem)
        spk_rec.append(out)

    mem_rec = torch.stack(mem_rec,dim=1)
    spk_rec = torch.stack(spk_rec,dim=1)

    # Readout layer
    h2= torch.einsum("abc,cd->abd", (spk_rec, w2))
    flt = torch.zeros((batch_size,nb_outputs), device=device, dtype=dtype)
    out = torch.zeros((batch_size,nb_outputs), device=device, dtype=dtype)
    out_rec = [out]
    for t in range(nb_steps):
        new_flt = alpha*flt +h2[:,t]
        new_out = beta*out +flt

        flt = new_flt
        out = new_out

        out_rec.append(out)

    out_rec = torch.stack(out_rec,dim=1)
    other_recs = [mem_rec, spk_rec]
    return out_rec, other_recs

In [ ]:
def train(lr=2e-3, nb_epochs=10, loss_hist=None):
    params = [w1,w2]
    optimizer = torch.optim.Adam(params, lr=lr, betas=(0.9,0.999))

    log_softmax_fn = nn.LogSoftmax(dim=1)
    loss_fn = nn.NLLLoss()
    
    if loss_hist is None:
        loss_hist = []
    for e in range(nb_epochs):
        print_counter = 0
        local_loss = []
        for x_data, y_data in from_cache_generator():
            if print_counter % 4 == 0:
                print '.',
            print_counter += 1
            
            y_data = y_data.astype(np.long)
            max_count = x_data.shape[0] / batch_size
            for counter in range(max_count):
                i = counter*batch_size
                x_local = torch.from_numpy(x_data[i:i+batch_size, :, :]).type(dtype)
                y_local = torch.from_numpy(y_data[i:i+batch_size])

                if x_local.shape[0] != batch_size:
                    continue
                
                output,_ = run_snn(x_local)
                m,_=torch.max(output,1)
                log_p_y = log_softmax_fn(m)
                loss_val = loss_fn(log_p_y, y_local)

                optimizer.zero_grad()
                loss_val.backward()
                optimizer.step()
                local_loss.append(loss_val.item())
        mean_loss = np.mean(local_loss)
        print("Epoch %i: loss=%.5f"%(e+1,mean_loss))
        loss_hist.append(mean_loss)
        
    return loss_hist


In [ ]:
%matplotlib inline
loss_hist = train(lr=2e-4, nb_epochs=10, loss_hist=loss_hist)

plt.figure(figsize=(3.3,2),dpi=150)
plt.plot(loss_hist)
plt.xlabel("Epoch")
plt.ylabel("Loss")
sns.despine()

In [30]:
def compute_classification_accuracy(x_data, y_data):
    """ Computes classification accuracy on supplied data in batches. """
    accs = []
    max_count = len(x_data) / batch_size
    for counter in range(max_count):
        i = counter*batch_size
        x_local = torch.from_numpy(x_data[i:i+batch_size, :, :]).type(dtype)
        y_local = torch.from_numpy(y_data[i:i+batch_size])
        output, _ = run_snn(x_local)
        m, _ = torch.max(output,1) # max over time
        _, am = torch.max(m,1)      # argmax over output units
        tmp = np.mean((y_local.type(torch.long)==am).detach().cpu().numpy()) # compare to labels
        accs.append(tmp)
    return np.mean(accs)

errors = []
for x_data, y_data in from_cache_generator():
    errors.append(compute_classification_accuracy(x_data, y_data))
print("Train accuracy: %.3f" % (np.mean(np.array(errors))))

errors = []
for x_data, y_data in from_cache_generator(False):
    errors.append(compute_classification_accuracy(x_data, y_data))
print("Test accuracy: %.3f" % (np.mean(np.array(errors))))

Train accuracy: 0.856
Test accuracy: 0.863
